# Creating Inputs for Network Visualization in D3.js and Gephi
## Using NIST National Vulnerabilities Database

### Importing the Necessary Packages into Python for processing

We first import the necessary packages to process the NIST National Vulnerabilities Database (NVD) for 2015. The xml file is available here: [NIST NVD Website](https://nvd.nist.gov/download.cfm#CVE_FEED). The packages used are:
    1. re for parsing the xml with regex
    2. csv for exporting to comma separated variable file
    3. json for exporting to JSON
    4. datetime for determining week number of year and day number of week from timestamp
    5. (Optional) tqdm for creating a cheap progress bar on an iterable.

In [ ]:
import re, csv, json, datetime
from tqdm import *

### Defining a function to extract necessary information

We first define a function that can parse the NIST NVD xml file and extract the relevant information. We parse out more fields than is needed for our visualisations. The additional information can be used in the future to create more insights into the NIST NVD. The function returns a dictionary by vulnerability key as defined by https://cve.mitre.org/. All additional information is then stored in a dictionary for that particular vulnerability.

In [ ]:
def pull_nvd(fname):
    print 'Loading NVD Dataset: %s'%fname
    nvd_dict = {}
    with open(fname) as f:
        for ln in tqdm(f):
            if 'entry id=' in ln:
                vuln_id = re.search('CVE.\d\d\d\d.\d\d\d\d', ln).group(0)
                # print '************************%s************************'%vuln_id
                nvd_dict[vuln_id] = {}
            elif 'cpe-lang:fact-ref name=' in ln:
                nvd_dict[vuln_id]['vuln_os'] = []
                nvd_dict[vuln_id]['vuln_os_plat'] = []
                vuln_os = re.search('\".*\"', ln).group(0)[8:-1]
                vuln_os_plat = re.split('[:\"]', ln)
                # print '--->%s'%vuln_os
                nvd_dict[vuln_id]['vuln_os'].append(re.sub('_',' ',vuln_os_plat[4]))
                nvd_dict[vuln_id]['vuln_os_plat'].append(re.sub('_',' ',vuln_os_plat[4])+' '+re.sub('_',' ',vuln_os_plat[5]))
            elif '<vuln:published-datetime>' in ln:
                vuln_date = re.search('\d\d\d\d.*<', ln).group(0)[:-1]
                date = [int(vuln_date[0:4]),int(vuln_date[5:7]),int(vuln_date[8:10]), int(vuln_date[11:13]), int(vuln_date[14:16]), int(vuln_date[17:19])]
                date_num = datetime.date(date[0],date[1],date[2]).isocalendar()
                datetime_num = datetime.datetime(date[0],date[1],date[2],date[3],date[4],date[5]).strftime('%Y%m%d %X')
                # print '    --->%s'%vuln_date
                nvd_dict[vuln_id]['vuln_date'] = date_num
                nvd_dict[vuln_id]['vuln_datetime'] = datetime_num
            elif '<cvss:score>' in ln:
                vuln_score = re.search('\d?\d.\d', ln).group(0)
                # print '    --->%s'%vuln_score
                nvd_dict[vuln_id]['vuln_score'] = vuln_score
            elif '<cvss:access-vector>' in ln:
                vuln_vector = re.search('>.*<', ln).group(0)[1:-1]
                # print '    --->%s'%vuln_vector
                nvd_dict[vuln_id]['vuln_vector'] = vuln_vector
            elif '<cvss:access-complexity>' in ln:
                vuln_compl = re.search('>.*<', ln).group(0)[1:-1]
                # print '    --->%s'%vuln_compl
                nvd_dict[vuln_id]['vuln_compl'] = vuln_compl
            elif '<cvss:authentication>' in ln:
                vuln_auth = re.search('>.*<', ln).group(0)[1:-1]
                # print '    --->%s'%vuln_auth
                nvd_dict[vuln_id]['vuln_auth'] = vuln_auth
            elif '<cvss:confidentiality-impact>' in ln:
                vuln_confid = re.search('>.*<', ln).group(0)[1:-1]
                # print '    --->%s'%vuln_confid
                nvd_dict[vuln_id]['vuln_confid'] = vuln_confid
            elif '<cvss:integrity-impact>' in ln:
                vuln_integ = re.search('>.*<', ln).group(0)[1:-1]
                # print '    --->%s'%vuln_integ
                nvd_dict[vuln_id]['vuln_integ'] = vuln_integ
            elif '<cvss:availability-impact>' in ln:
                vuln_avail = re.search('>.*<', ln).group(0)[1:-1]
                # print '    --->%s'%vuln_avail
                nvd_dict[vuln_id]['vuln_avail'] = vuln_avail
            elif 'vuln:reference href=' in ln:
                vuln_link = re.search('href=.*\"', ln).group(0)[6:-1]
                # print '    --->%s'%vuln_link
                nvd_dict[vuln_id]['vuln_link'] = vuln_link
            elif '<vuln:summary>' in ln:
                vuln_summ = re.search('>.*<', ln).group(0)[1:-1]
                # print '    --->%s'%vuln_summ
                nvd_dict[vuln_id]['vuln_summ'] = vuln_summ
            elif '<vuln:source>' in ln:
                vuln_source = re.search('>.*<', ln).group(0)[1:-1]
                # print '    --->%s'%vuln_summ
                nvd_dict[vuln_id]['vuln_source'] = vuln_source
    return(nvd_dict)

### Run the function we defined

Now run the defined function on the NVD 2015 xml file extracted to the working directory.

In [ ]:
nvd = pull_nvd('./nvdcve-2.0-2015.xml')

### Create a timeline for 2015 so far

We create a timeline for 2015 so far by count for both the unweighted instances and the weighted totals given the CVSS score provided. Both the amount of vulnerabilities reported and the evaluated threat they pose on that day is of interest. We compute a running sum for both to see how much of a difference there is day to day.

In [ ]:
nvd_timeline = [[],[]]
for cve in tqdm(nvd.keys()):
    if 'vuln_score' in nvd[cve].keys():
        weightedme = {'Method':'Weighted', 'Date':nvd[cve]['vuln_datetime'], 'Value':float(nvd[cve]['vuln_score'])}
        countme = {'Method':'Count', 'Date':nvd[cve]['vuln_datetime'], 'Value':1}
        if nvd[cve]['vuln_datetime'] not in [ x['Date'] for x in nvd_timeline[0] ]:
            nvd_timeline[0].append(weightedme)
            nvd_timeline[1].append(countme)
        else:
            for i, day in enumerate(nvd_timeline[0]):
                if nvd[cve]['vuln_datetime'] in day['Date'] and day['Method'] == 'Weighted':
                    nvd_timeline[0][i]['Value'] += float(nvd[cve]['vuln_score'])
                elif nvd[cve]['vuln_datetime'] in day['Date'] and day['Method'] == 'Count':
                    nvd_timeline[1][i]['Value'] += 1

Export our newly structured data to the local working directory as a JSON. This is now an input for our d3.js + dimple.js scatter plot.

In [ ]:
with open('./nvd_timeline.json', 'wb') as f:
    json.dump(nvd_timeline, f)

### Create a count of vulnerabilities per platform

Now we are interested in the number of vulnerabilities reported by platform. Does conventional wisdom hold up? Does Apple receive less reported vulnerabilities than Microsoft? Does Linux receive even less? To answer this, we create a new entry for each new platform and do a running sum as we parse through our defined dictionary. We then sort the resulting list and add a header to make loading in javascript faster.

In [ ]:
nvd_plat = []
for cve in tqdm(nvd.keys()):
    if 'vuln_score' in nvd[cve].keys():
        for os in nvd[cve]['vuln_os']:
            appendme = [os, 1]
            if os not in [ x[0] for x in nvd_plat ]:
                nvd_plat.append(appendme)
            else:
                for i, plat in enumerate(nvd_plat):
                    if os == plat[0]:
                        nvd_plat[i][1] += 1

nvd_plat_sort = [['plat','count']]
nvd_plat_sort.extend(sorted(nvd_plat))

Export our (again) newly structured data this time to a CSV in the local working directory. We show how different file types can be exported from Python. We can then use our exported file as an input for d3.js + dimple.js histogram.

In [ ]:
with open('./nvd_plat.csv', 'wb') as csvfile:
    writeme = csv.writer(csvfile, delimiter=',')
    writeme.writerows(nvd_plat_sort)

### Create network graph with weekly information

Now we want to look at what platforms are being attacked for each week. We do this by creating a dictionary of nodes and edges with the additional week number, day number, and threat score loaded. Then given the dictionary, export to JSON file.

In [ ]:
nvd_json = {'nodes':[],'links':[]}
for cve in nvd.keys():
    if 'vuln_os' in nvd[cve].keys():
        append_vuln = {'name':cve, 'group':nvd[cve]['vuln_date'][2], 'week':nvd[cve]['vuln_date'][1], 'threat':float(nvd[cve]['vuln_score']), 'type':1}
        nvd_json['nodes'].append(append_vuln)
        for os in nvd[cve]['vuln_os']:
            append_plat = {'name':os, 'group':nvd[cve]['vuln_date'][2], 'week':nvd[cve]['vuln_date'][1], 'threat':0, 'type':0}
            if append_plat not in nvd_json['nodes']:
                nvd_json['nodes'].append(append_plat)
            nvd_json['links'].append({'source':cve, 'target':os, 'group':nvd[cve]['vuln_date'][2], 'week':nvd[cve]['vuln_date'][1], 'threat':float(nvd[cve]['vuln_score'])})

We now export to JSON for our d3.js network graph.

In [ ]:
with open('./nvd.json', 'wb') as f:
    json.dump(nvd_json, f)

### Build inputs for Gephi (Big Annual Network Graph)

Weekly slices are important but we also want to see the whole network of vulnerabilities and platforms for 2015 at once. The number of nodes (platforms and vulnerabilities) and edges are too big for an interactive d3.js visualization. We instead use [Gephi](http://gephi.github.io/). Gephi requires an edge list and (optionally) a node list (if unconnected nodes exist). We create both in this exercise as list arrays with headers filled in. Then populate each row with either the edge or node. Finally, writing both files to csv files in working directory.

In [ ]:
nvd_edge = [['source','target','value']]
for cve in nvd.keys():
    if 'vuln_score' in nvd[cve].keys():
        threat = nvd[cve]['vuln_score']
    else:
        threat = 0.0
    if 'vuln_os' in nvd[cve].keys():
        cve_type = (nvd[cve]['vuln_vector'], nvd[cve]['vuln_compl'], nvd[cve]['vuln_auth'], nvd[cve]['vuln_confid'], nvd[cve]['vuln_integ'])
        for plat in nvd[cve]['vuln_os']:
            appendme = [cve_type, plat, threat]
            if appendme not in nvd_edge:
                nvd_edge.append(appendme)

In [ ]:
nvd_node = [['id','value','type']]
for row in nvd_edge:
    append_vuln = [row[0],row[2],0]
    append_targ = [row[1],0.0,1]
    if append_vuln not in nvd_node:
        nvd_node.append(append_vuln)
    if append_targ not in nvd_node:
        nvd_node.append(append_targ)

We now export them to CSV for Gephi.

In [ ]:
with open('./nvd_edge.csv', 'wb') as csvfile:
    writeme = csv.writer(csvfile, delimiter=',')
    writeme.writerows(nvd_edge)

with open('./nvd_node.csv', 'wb') as csvfile:
    writeme = csv.writer(csvfile, delimiter=',')
    writeme.writerows(nvd_node)

### Looking at other years of the NVD

Given our look at NVD in 2015, we want to expand our look to past years. We do this by expanding our original dictionary to go beyond 2015. We load previous years of the NVD with the same function and update the original dictionary we defined. Then count for each week how many vulnerabilities are reported for each year.

In [ ]:
nvd.update(pull_nvd('./nvdcve-2.0-2011.xml'))
nvd.update(pull_nvd('./nvdcve-2.0-2012.xml'))
nvd.update(pull_nvd('./nvdcve-2.0-2013.xml'))
nvd.update(pull_nvd('./nvdcve-2.0-2014.xml'))

Now with our newly beefed up dictionary that includes 2011 to 2015 datasets from NVD, we start processing to create time series for all five years. To do this we look at iso week number for any given year and conducting a running sum of the number of vulnerabilities reported. Commented out is an alternative look at the combined CVSS score for that week.

In [ ]:
nvd_timeline_five = []
for cve in tqdm(nvd.keys()):
    if 'vuln_score' in nvd[cve].keys():
        appendme = [nvd[cve]['vuln_date'][1],0,0,0,0,0]
        if nvd[cve]['vuln_date'][0] == 2015:
            colnum = 1
        elif nvd[cve]['vuln_date'][0] == 2014:
            colnum = 2
        elif nvd[cve]['vuln_date'][0] == 2013:
            colnum = 3
        elif nvd[cve]['vuln_date'][0] == 2012:
            colnum = 4
        elif nvd[cve]['vuln_date'][0] == 2011:
            colnum = 5
        if nvd[cve]['vuln_date'][1] not in [ x[0] for x in nvd_timeline_five ]:
            # appendme[colnum] += float(nvd[cve]['vuln_score'])
            appendme[colnum] += 1
            nvd_timeline_five.append(appendme)
        else:        
            for i, day in enumerate(nvd_timeline_five):
                if nvd[cve]['vuln_date'][1] == day[0]:
                    # nvd_timeline_five[i][colnum] += float(nvd[cve]['vuln_score'])
                    nvd_timeline_five[i][colnum] += 1

Like before we want to sort the resulting structured data and attach a header to it. This reduces processing on the javascript end.

In [ ]:
nvd_timeline_five_sort = [['week','2015','2014','2013','2012','2011']]
nvd_timeline_five_sort.extend(sorted(nvd_timeline_five))

Now we can export it to a CSV for our dygraph.js plot.

In [ ]:
with open('./nvd_timeline_five.csv', 'wb') as csvfile:
    writeme = csv.writer(csvfile, delimiter=',')
    writeme.writerows(nvd_timeline_five_sort)